# Popular first names by municipality

This is an ETL process for creating a datamodel for reporting popular first names and their distribution across municipalities in Finland.

The code is divided into 3 parts: creation of the **municipality dimension**, creation of the **first name amounts fact** and bonus assignment on the **relative frequency distribution**.

Source file formats consist of JSONL and Apache Parquet. They're quickly parsed to the common data structures in order to decouple the code from the file type. As target database, **SQLite** was chosen for the demonstration, so that no separate database installation is needed. The code is written in **Python 3**, more specifically 3.8.15.

Since there was no specification on the language and the assignment was in English, I have chosen to write the code, the data model and descriptions in **English**. Modifications to the data (gender) are in **Finnish**, because most of the source data was in Finnish.

In [1]:
import pyarrow.parquet as parquet
import sqlite3
import datetime
import jsonlines
import pandas

In [2]:
parquetUrl = 'data/municipality-listing-2020-10-21.parquet'
jsonlUrl = 'data/most-popular-first-names-by-municipality.jsonl'

## Municipality (dimension)

The municipalities source data was already cleaned, so only metadata was added.

In [3]:
municipalities = parquet.read_table(parquetUrl).to_pylist()

print(municipalities[0])

{'MUNICIPALITY_NUMBER': 5, 'MUNICIPALITY_NAME_FI': 'Alajärvi', 'MUNICIPALITY_NAME_SE': 'Alajärvi', 'MUNICIPALITY_TYPE': 'Kaupunki', 'PRIMARY_LANGUAGE': 'suomi', 'ELECTORAL_DISTRICT_NUMBER': 10, 'ELECTORAL_DISTRICT_NAME_FI': 'Vaasan vaalipiiri', 'ELECTORAL_DISTRICT_NAME_SE': 'Vasa valkrets', 'REGION_NUMBER': 13, 'REGION_NAME_FI': 'Etelä-Pohjanmaa', 'REGION_NAME_SE': 'Södra Österbotten'}


In [4]:
for i in range(0,len(municipalities)):
    municipalities[i]['ID'] = i+1

loaddate = datetime.datetime.now()
for i in range(0,len(municipalities)):
    municipalities[i]['LOAD_DATE'] = loaddate

print(municipalities[0])

{'MUNICIPALITY_NUMBER': 5, 'MUNICIPALITY_NAME_FI': 'Alajärvi', 'MUNICIPALITY_NAME_SE': 'Alajärvi', 'MUNICIPALITY_TYPE': 'Kaupunki', 'PRIMARY_LANGUAGE': 'suomi', 'ELECTORAL_DISTRICT_NUMBER': 10, 'ELECTORAL_DISTRICT_NAME_FI': 'Vaasan vaalipiiri', 'ELECTORAL_DISTRICT_NAME_SE': 'Vasa valkrets', 'REGION_NUMBER': 13, 'REGION_NAME_FI': 'Etelä-Pohjanmaa', 'REGION_NAME_SE': 'Södra Österbotten', 'ID': 1, 'LOAD_DATE': datetime.datetime(2023, 2, 13, 1, 2, 25, 665334)}


In [5]:
conn = sqlite3.connect('first_name_amounts.sqlite')

In [6]:
conn.execute('''
            drop table if exists municipalities;
            ''')
conn.execute('''
            create table municipalities (
            id integer primary key
            , municipality_number integer
            , municipality_name_fi text
            , municipality_name_se text
            , municipality_type text
            , primary_language text
            , electoral_district_number integer
            , electoral_district_name_fi text
            , electoral_district_name_se text
            , region_number integer
            , region_name_fi text
            , region_name_se text
            , load_date text
            );
            ''')

In [7]:
for mp in municipalities:
    conn.execute('''
        insert into municipalities (
        id
        , municipality_number
        , municipality_name_fi
        , municipality_name_se
        , municipality_type
        , primary_language
        , electoral_district_number
        , electoral_district_name_fi
        , electoral_district_name_se
        , region_number
        , region_name_fi
        , region_name_se
        , load_date
        ) values
        (?,?,?,?,?,?,?,?,?,?,?,?,?);
        ''',
        (
            mp['ID'],
            mp['MUNICIPALITY_NUMBER'],
            mp['MUNICIPALITY_NAME_FI'],
            mp['MUNICIPALITY_NAME_SE'],
            mp['MUNICIPALITY_TYPE'],
            mp['PRIMARY_LANGUAGE'],
            mp['ELECTORAL_DISTRICT_NUMBER'],
            mp['ELECTORAL_DISTRICT_NAME_FI'],
            mp['ELECTORAL_DISTRICT_NAME_SE'],
            mp['REGION_NUMBER'],
            mp['REGION_NAME_FI'],
            mp['REGION_NAME_SE'],
            mp['LOAD_DATE']
        ))
conn.commit()
conn.close()

## First name amounts (fact)

The aggregated fact table is loaded from a JSONL file and needed a bit of clean up.

Municipality information is moved into a single reference to the dimension. This required applying a few assumptions which are explained below.

In [8]:
fact = []
with jsonlines.open(jsonlUrl) as reader:
    for row in reader:
        fact.append(row)
        
fact[0]

{'YEAR_OF_BIRTH': '1969',
 'BIRTH_MUNICIPALITY_NUMBER': '049',
 'BIRTH_MUNICIPALITY_NAME': 'Espoo',
 'GENDER': '1',
 'FIRST_NAME': 'Mika',
 'AMOUNT': '39'}

In [9]:
for i in range(0,len(fact)):
    fact[i]['ID'] = i+1

print(fact[0])

{'YEAR_OF_BIRTH': '1969', 'BIRTH_MUNICIPALITY_NUMBER': '049', 'BIRTH_MUNICIPALITY_NAME': 'Espoo', 'GENDER': '1', 'FIRST_NAME': 'Mika', 'AMOUNT': '39', 'ID': 1}


For the mapping of the municipality to the dimension, it is assumed that the municipality name represent the municipality. So, if the municipality number is missing or an older municipality number is used, that doesn't mean there has been 2 municipalities with the same name, but these both refer to the same entity. And thus both of these cases can be mapped to the row with the new municipality number.

In [10]:
def findMunicipalityIdbyName(municipality_name, municipalities):
    if(municipality_name is None):
        return -1
    mun_id = -1
    for row in municipalities:
        if row['MUNICIPALITY_NAME_FI'] == municipality_name:
            mun_id = row['ID']
    return mun_id

In [11]:
def findMunicipalityId(municipality_number, municipalities):
    if(municipality_number is None):
        return -1
    mun_id = -1
    for row in municipalities:
        if row['MUNICIPALITY_NUMBER'] == int(municipality_number):
            mun_id = row['ID']
    return mun_id

In [12]:
for row in fact:
    if row['BIRTH_MUNICIPALITY_NUMBER'] is None:
        row['MUNICIPALITY_ID'] = findMunicipalityIdbyName(row['BIRTH_MUNICIPALITY_NAME'], municipalities)
    else:
        row['MUNICIPALITY_ID'] = findMunicipalityId(row['BIRTH_MUNICIPALITY_NUMBER'], municipalities)

for row in fact:
    if row['MUNICIPALITY_ID'] == -1:
        row['MUNICIPALITY_ID'] = findMunicipalityIdbyName(row['BIRTH_MUNICIPALITY_NAME'], municipalities)

Here Finnish is used in order to stay consistent with the language used elsewhere in the data.

In [13]:
for row in fact:
    if row['GENDER'] == '1':
        row['GENDER'] = 'Mies'
    else:
        row['GENDER'] = 'Nainen'

Here I realized that the Pandas DataFrame could've been much easier way to approach this. With some extra time, I would next go to unify earlier steps to use pandas.

In [14]:
df = pandas.DataFrame(fact)

These rows have no first name and no amount. Without extra information from the source, these provide no value to the data model.

In [15]:
df = df.dropna(subset=['FIRST_NAME'])

Here I utilized the same functionalities as in the dimensio, so I had to convert back from pandas.

In [16]:
fact = df.to_dict('records')
fact[0]

{'YEAR_OF_BIRTH': '1969',
 'BIRTH_MUNICIPALITY_NUMBER': '049',
 'BIRTH_MUNICIPALITY_NAME': 'Espoo',
 'GENDER': 'Mies',
 'FIRST_NAME': 'Mika',
 'AMOUNT': '39',
 'ID': 1,
 'MUNICIPALITY_ID': 12}

Pandas conversion changes the datatype, so this is executed after the conversions.

In [17]:
loaddate = datetime.datetime.now()
for i in range(0,len(fact)):
    fact[i]['LOAD_DATE'] = loaddate

fact[0]

{'YEAR_OF_BIRTH': '1969',
 'BIRTH_MUNICIPALITY_NUMBER': '049',
 'BIRTH_MUNICIPALITY_NAME': 'Espoo',
 'GENDER': 'Mies',
 'FIRST_NAME': 'Mika',
 'AMOUNT': '39',
 'ID': 1,
 'MUNICIPALITY_ID': 12,
 'LOAD_DATE': datetime.datetime(2023, 2, 13, 1, 2, 26, 499101)}

In [18]:
conn = sqlite3.connect('first_name_amounts.sqlite')

In [19]:
conn.execute('''
            drop table if exists first_name_amounts;
            ''')
conn.execute('''
            create table first_name_amounts (
                  id integer primary key
                , first_name text
                , gender text
                , municipality_id integer
                , year_of_birth integer
                , amount integer
                , load_date text
            );
            ''')

In [20]:
for row in fact:
    conn.execute('''
        insert into first_name_amounts (
              id
            , first_name
            , gender
            , municipality_id
            , year_of_birth
            , amount
            , load_date
        ) values
        (?,?,?,?,?,?,?);
        ''',
        (
            row['ID'],
            row['FIRST_NAME'],
            row['GENDER'],
            row['MUNICIPALITY_ID'],
            row['YEAR_OF_BIRTH'],
            row['AMOUNT'],
            row['LOAD_DATE']
        ))
conn.commit()
conn.close()

## Relative frequency distribution

Assignment: "What is the relative frequency distribution over time (as percentages) for the most popular male names in the cities ('Kaupunki') of the region 'Pohjois-Savo'?"

In [21]:
conn = sqlite3.connect('first_name_amounts.sqlite')

df = pandas.read_sql_query('''
                            select
                                  fna.year_of_birth as year
                                , fna.first_name as name
                                , round((100.0 *sum(fna.amount)) / sum(sum(fna.amount)) over (partition by fna.year_of_birth),2) || '%' as rel_freq_distr
                            from
                                first_name_amounts fna
                            join
                                municipalities mu
                            on fna.municipality_id = mu.id
                            where
                                mu.region_name_fi = 'Pohjois-Savo' and
                                mu.municipality_type = 'Kaupunki' and
                                fna.gender = 'Mies'
                            group by fna.year_of_birth, fna.first_name
                            order by fna.year_of_birth, fna.first_name
                            ;
                            ''', conn)

conn.close()

In [22]:
pandas.set_option('display.max_rows', None)
df

,year,name,rel_freq_distr
0,1969,Marko,19.12%
1,1969,Mika,80.88%
2,1970,Mika,100.0%
3,1971,Janne,10.78%
4,1971,Jari,10.78%
5,1971,Marko,20.59%
6,1971,Mika,57.84%
7,1972,Marko,43.43%
8,1972,Mika,30.3%
9,1972,Pasi,13.13%
